In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.imports import *

from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
#import skimage

In [2]:
from planet import f2

In [3]:
# major back groun color

In [4]:
PATH = "/home/embsysa/fastai/pytorch_classifiers-master/data/train_py_theme/"
sz = 224
arch = resnet34
bs = 24

In [10]:
m = arch(True)

In [11]:
m = nn.Sequential(*children(m)[:-2], 
                  nn.Conv2d(512,3 , 3, padding=1), 
                  nn.AdaptiveAvgPool2d(1), Flatten(), 
                  nn.LogSoftmax())

In [12]:
m

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, ker

In [13]:
tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

In [14]:
learn = ConvLearner.from_model_data(m, data)
#learn = ConvLearner.pretrained(arch, data)

In [18]:
learn.freeze_to(-8)

In [19]:
learn.fit(1e-5, 100)

epoch      trn_loss   val_loss   accuracy                    
    0      0.944939   1.028722   0.510274  
    1      0.949646   1.016367   0.520548                    
    2      0.931658   1.01972    0.526256                    
    3      0.928547   1.010656   0.517123                    
    4      0.897588   1.01342    0.52169                     
    5      0.899098   1.006913   0.520548                    
    6      0.903031   0.998589   0.520548                    
    7      0.885265   0.985484   0.52169                     
    8      0.869913   0.988261   0.52169                     
    9      0.892283   0.987421   0.523973                    
    10     0.861777   0.98498    0.528539                    
    11     0.863187   0.983603   0.530822                    
    12     0.855508   0.983348   0.533105                    
    13     0.84401    0.987785   0.52968                     
    14     0.845451   0.983004   0.525114                    
    15     0.841644   0.97

KeyboardInterrupt: 

In [17]:
learn.save('shallow_majorBackgroundColor')

In [ ]:
learn.fit(1e-3, 25, cycle_len=1)

## Class Activation Maps (CAM)

In [ ]:
class SaveFeatures():
    features=None
    def __init__(self, m): self.hook = m.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output): self.features = output
    def remove(self): self.hook.remove()

In [ ]:
x,y = next(iter(data.val_dl))

In [ ]:
x,y = x[None,1], y[None,1]
vx = Variable(x.cuda(), requires_grad=True)

In [ ]:
dx = data.val_ds.denorm(x)[0]
plt.imshow(dx);

In [ ]:
sfs = [SaveFeatures(o) for o in [m[-7], m[-6], m[-5], m[-4]]]

In [ ]:
%time py = m(Variable(x.cuda()))

In [ ]:
for o in sfs: o.remove()

In [ ]:
[o.features.size() for o in sfs]

In [ ]:
py = np.exp(to_np(py)[0]); py

In [ ]:
feat = np.maximum(0,to_np(sfs[3].features[0]))
feat.shape

In [ ]:
f2=np.dot(np.rollaxis(feat,0,3), py)
f2-=f2.min()
f2/=f2.max()
f2

In [ ]:
plt.imshow(dx)
plt.imshow(skimage.transform.resize(f2, dx.shape), alpha=0.5, cmap='hot');

## Model

In [ ]:
learn.unfreeze()
learn.bn_freeze(True)

In [ ]:
# 12 layer groups call for 12 lrs
lr=np.array([[1e-6]*4,[1e-4]*4,[1e-2]*4]).flatten()

In [ ]:
learn.fit(lr, 2, cycle_len=1)

In [ ]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),0)
accuracy_np(preds,y)

In [ ]:
learn.fit(lr, 2, cycle_len=1)

In [ ]:
log_preds,y = learn.TTA()
preds = np.mean(np.exp(log_preds),0)
accuracy_np(preds,y)